In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import seaborn as sns
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from xgboost import XGBRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [2]:
df = pd.read_csv(r"C:\Users\Ben\Desktop\moviedata\proj2train.csv", header=0) 
df.head()
test = pd.read_csv(r"C:\Users\Ben\Desktop\moviedata\proj2test.csv")

In [3]:
test.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew
0,3001,"[{'id': 34055, 'name': 'Pokémon Collection', '...",0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",http://www.pokemon.com/us/movies/movie-pokemon...,tt1226251,ja,ディアルガVSパルキアVSダークライ,Ash and friends (this time accompanied by newc...,3.851534,...,"[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",7/14/07,90.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Somewhere Between Time & Space... A Legend Is ...,Pokémon: The Rise of Darkrai,"[{'id': 11451, 'name': 'pok√©mon'}, {'id': 115...","[{'cast_id': 3, 'character': 'Tonio', 'credit_...","[{'credit_id': '52fe44e7c3a368484e03d683', 'de..."
1,3002,NaN,88000,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",NaN,tt0051380,en,Attack of the 50 Foot Woman,When an abused wife grows to giant size becaus...,3.559789,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",5/19/58,65.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A titanic beauty spreads a macabre wave of hor...,Attack of the 50 Foot Woman,"[{'id': 9748, 'name': 'revenge'}, {'id': 9951,...","[{'cast_id': 2, 'character': 'Nancy Fowler Arc...","[{'credit_id': '55807805c3a3685b1300060b', 'de..."
2,3003,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,tt0118556,en,Addicted to Love,Good-natured astronomer Sam is devastated when...,8.085194,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",5/23/97,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A Comedy About Lost Loves And Last Laughs,Addicted to Love,"[{'id': 931, 'name': 'jealousy'}, {'id': 9673,...","[{'cast_id': 11, 'character': 'Maggie', 'credi...","[{'credit_id': '52fe4330c3a36847f8041367', 'de..."
3,3004,NaN,6800000,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",http://www.sonyclassics.com/incendies/,tt1255953,fr,Incendies,A mother's last wishes send twins Jeanne and S...,8.596012,...,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",9/4/10,130.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,The search began at the opening of their mothe...,Incendies,"[{'id': 378, 'name': 'prison'}, {'id': 539, 'n...","[{'cast_id': 6, 'character': 'Nawal', 'credit_...","[{'credit_id': '56478092c3a36826140043af', 'de..."
4,3005,NaN,2000000,"[{'id': 36, 'name': 'History'}, {'id': 99, 'na...",NaN,tt0418753,en,Inside Deep Throat,"In 1972, a seemingly typical shoestring budget...",3.217680,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",2/11/05,92.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It was filmed in 6 days for 25 thousand dollar...,Inside Deep Throat,"[{'id': 279, 'name': 'usa'}, {'id': 1228, 'nam...","[{'cast_id': 1, 'character': 'Narrator (voice)...","[{'credit_id': '52fe44ce9251416c75041967', 'de..."


In [4]:
df['year'] = pd.DatetimeIndex(df['release_date']).year

In [5]:
df['revenue'] = np.log1p(df['revenue'])

In [6]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [7]:
missing_values_table(df)

Your selected dataframe has 24 columns.
There are 13 columns that have missing values.


,Missing Values,% of Total Values
belongs_to_collection,2396,79.9
homepage,2054,68.5
tagline,597,19.9
Keywords,276,9.2
production_companies,156,5.2
production_countries,55,1.8
spoken_languages,20,0.7
crew,16,0.5
cast,13,0.4
overview,8,0.3


In [8]:
#df["has_colection"] = df['belongs_to_collection'].apply(lambda x: lambda: len(x) if x != {} else 0)

In [9]:
df.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue,year
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",16.326300,2015
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",18.370959,2004
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",16.387512,2014
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16.588099,2012
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",15.182615,2009


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     3000 non-null   int64  
 1   belongs_to_collection  604 non-null    object 
 2   budget                 3000 non-null   int64  
 3   genres                 2993 non-null   object 
 4   homepage               946 non-null    object 
 5   imdb_id                3000 non-null   object 
 6   original_language      3000 non-null   object 
 7   original_title         3000 non-null   object 
 8   overview               2992 non-null   object 
 9   popularity             3000 non-null   float64
 10  poster_path            2999 non-null   object 
 11  production_companies   2844 non-null   object 
 12  production_countries   2945 non-null   object 
 13  release_date           3000 non-null   object 
 14  runtime                2998 non-null   float64
 15  spok

In [11]:
#df = df.dropna()
#df['belongs_to_collection'] = df['belongs_to_collection'].fillna(0)
df = df.drop(["homepage",'belongs_to_collection' ], axis = 1)
test = test.drop(['homepage', "belongs_to_collection"], axis = 1)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3000 non-null   int64  
 1   budget                3000 non-null   int64  
 2   genres                2993 non-null   object 
 3   imdb_id               3000 non-null   object 
 4   original_language     3000 non-null   object 
 5   original_title        3000 non-null   object 
 6   overview              2992 non-null   object 
 7   popularity            3000 non-null   float64
 8   poster_path           2999 non-null   object 
 9   production_companies  2844 non-null   object 
 10  production_countries  2945 non-null   object 
 11  release_date          3000 non-null   object 
 12  runtime               2998 non-null   float64
 13  spoken_languages      2980 non-null   object 
 14  status                3000 non-null   object 
 15  tagline              

In [13]:
df = df.dropna()

In [14]:
missing_values_table(df)

Your selected dataframe has 22 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2229 entries, 0 to 2999
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    2229 non-null   int64  
 1   budget                2229 non-null   int64  
 2   genres                2229 non-null   object 
 3   imdb_id               2229 non-null   object 
 4   original_language     2229 non-null   object 
 5   original_title        2229 non-null   object 
 6   overview              2229 non-null   object 
 7   popularity            2229 non-null   float64
 8   poster_path           2229 non-null   object 
 9   production_companies  2229 non-null   object 
 10  production_countries  2229 non-null   object 
 11  release_date          2229 non-null   object 
 12  runtime               2229 non-null   float64
 13  spoken_languages      2229 non-null   object 
 14  status                2229 non-null   object 
 15  tagline              

In [16]:
df['year'] = pd.DatetimeIndex(df['release_date']).year

In [17]:
Y = df["revenue"]

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2229 entries, 0 to 2999
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    2229 non-null   int64  
 1   budget                2229 non-null   int64  
 2   genres                2229 non-null   object 
 3   imdb_id               2229 non-null   object 
 4   original_language     2229 non-null   object 
 5   original_title        2229 non-null   object 
 6   overview              2229 non-null   object 
 7   popularity            2229 non-null   float64
 8   poster_path           2229 non-null   object 
 9   production_companies  2229 non-null   object 
 10  production_countries  2229 non-null   object 
 11  release_date          2229 non-null   object 
 12  runtime               2229 non-null   float64
 13  spoken_languages      2229 non-null   object 
 14  status                2229 non-null   object 
 15  tagline              

In [19]:
Y.head()

0    16.326300
1    18.370959
2    16.387512
8    17.351454
9    16.746717
Name: revenue, dtype: float64

In [20]:
#df["IsCollection"].value_counts()

In [21]:
df.head()

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,...,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue,year
0,1,14000000,"[{'id': 35, 'name': 'Comedy'}]",tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...",...,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",16.326300,2015
1,2,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]",...,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",18.370959,2004
2,3,3300000,"[{'id': 18, 'name': 'Drama'}]",tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...",...,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",16.387512,2014
8,9,0,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",tt0117110,en,Muppet Treasure Island,After telling the story of Flint's last journe...,6.902423,/5A8gKzOrF9Z7tSUX6xd5dEx4NXf.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...",...,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Set sail for Muppet mayhem!,Muppet Treasure Island,"[{'id': 2041, 'name': 'island'}, {'id': 4418, ...","[{'cast_id': 1, 'character': 'Long John Silver...","[{'credit_id': '52fe43c89251416c7501deb3', 'de...",17.351454,1996
9,10,6000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",tt0310281,en,A Mighty Wind,"In ""A Mighty Wind"", director Christopher Guest...",4.672036,/xwD4EsPOGLuKP50Q1gwqAsyhAFB.jpg,"[{'name': 'Castle Rock Entertainment', 'id': 97}]",...,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"Back together for the first time, again.",A Mighty Wind,"[{'id': 11800, 'name': 'mockumentary'}, {'id':...","[{'cast_id': 24, 'character': 'Jonathan Steinb...","[{'credit_id': '52fe45609251416c750545b3', 'de...",16.746717,2003


In [22]:
df["budget_year_ratio"] = df['budget'] / (df['year'] * df['year'])

In [23]:
df.head()

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,...,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue,year,budget_year_ratio
0,1,14000000,"[{'id': 35, 'name': 'Comedy'}]",tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",16.326300,2015,3.448085
1,2,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",18.370959,2004,9.960120
2,3,3300000,"[{'id': 18, 'name': 'Drama'}]",tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",16.387512,2014,0.813570
8,9,0,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",tt0117110,en,Muppet Treasure Island,After telling the story of Flint's last journe...,6.902423,/5A8gKzOrF9Z7tSUX6xd5dEx4NXf.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Set sail for Muppet mayhem!,Muppet Treasure Island,"[{'id': 2041, 'name': 'island'}, {'id': 4418, ...","[{'cast_id': 1, 'character': 'Long John Silver...","[{'credit_id': '52fe43c89251416c7501deb3', 'de...",17.351454,1996,0.000000
9,10,6000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",tt0310281,en,A Mighty Wind,"In ""A Mighty Wind"", director Christopher Guest...",4.672036,/xwD4EsPOGLuKP50Q1gwqAsyhAFB.jpg,"[{'name': 'Castle Rock Entertainment', 'id': 97}]",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"Back together for the first time, again.",A Mighty Wind,"[{'id': 11800, 'name': 'mockumentary'}, {'id':...","[{'cast_id': 24, 'character': 'Jonathan Steinb...","[{'credit_id': '52fe45609251416c750545b3', 'de...",16.746717,2003,1.495510


In [24]:
def extractOverviewInfo(df):
    df['has_overview'] = df['overview'].apply(lambda x: isinstance(x, str))
    df['overview_len'] = df['overview'].apply(lambda x: len(x) if isinstance(x, str) else 0)
    return df

In [25]:
df = extractOverviewInfo(df)
df

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,...,tagline,title,Keywords,cast,crew,revenue,year,budget_year_ratio,has_overview,overview_len
0,1,14000000,"[{'id': 35, 'name': 'Comedy'}]",tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...",...,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",16.326300,2015,3.448085,True,155
1,2,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]",...,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",18.370959,2004,9.960120,True,393
2,3,3300000,"[{'id': 18, 'name': 'Drama'}]",tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...",...,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",16.387512,2014,0.813570,True,130
8,9,0,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",tt0117110,en,Muppet Treasure Island,After telling the story of Flint's last journe...,6.902423,/5A8gKzOrF9Z7tSUX6xd5dEx4NXf.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...",...,Set sail for Muppet mayhem!,Muppet Treasure Island,"[{'id': 2041, 'name': 'island'}, {'id': 4418, ...","[{'cast_id': 1, 'character': 'Long John Silver...","[{'credit_id': '52fe43c89251416c7501deb3', 'de...",17.351454,1996,0.000000,True,397
9,10,6000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",tt0310281,en,A Mighty Wind,"In ""A Mighty Wind"", director Christopher Guest...",4.672036,/xwD4EsPOGLuKP50Q1gwqAsyhAFB.jpg,"[{'name': 'Castle Rock Entertainment', 'id': 97}]",...,"Back together for the first time, again.",A Mighty Wind,"[{'id': 11800, 'name': 'mockumentary'}, {'id':...","[{'cast_id': 24, 'character': 'Jonathan Steinb...","[{'credit_id': '52fe45609251416c750545b3', 'de...",16.746717,2003,1.495510,True,280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2994,2995,0,"[{'id': 18, 'name': 'Drama'}]",tt0105327,en,School Ties,When David Greene receives a football scholars...,7.438381,/poV3j71mcmQkmjezc2H35xJsAhD.jpg,"[{'name': 'Paramount Pictures', 'id': 4}]",...,Just because you’re accepted doesn’t mean you ...,School Ties,"[{'id': 6075, 'name': 'sport'}, {'id': 10144, ...","[{'cast_id': 2, 'character': 'David Greene', '...","[{'credit_id': '5637777ac3a3681b4d01f9f5', 'de...",16.504383,1992,0.000000,True,435
2995,2996,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",tt0109403,en,Chasers,Military men Rock Reilly and Eddie Devane are ...,9.853270,/j8Q7pQ27hvH54wpxJzIuQgQCdro.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",...,It was supposed to be a routine prisoner trans...,Chasers,"[{'id': 378, 'name': 'prison'}, {'id': 572, 'n...","[{'cast_id': 2, 'character': 'Rock Reilly', 'c...","[{'credit_id': '52fe4494c3a368484e02ac7d', 'de...",14.283442,1994,0.000000,True,261
2997,2998,65000000,"[{'id': 80, 'name': 'Crime'}, {'id': 28, 'name...",tt0116908,en,The Long Kiss Goodnight,"Samantha Caine, suburban homemaker, is the ide...",14.482345,/4MENR8x6mYqnZvp2hGjSaPJz64J.jpg,"[{'name': 'New Line Cinema', 'id': 12}, {'name...",...,What's forgotten is not always gone.

In [26]:
df['has_overview'] = df['has_overview'].astype(int)
df

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,...,tagline,title,Keywords,cast,crew,revenue,year,budget_year_ratio,has_overview,overview_len
0,1,14000000,"[{'id': 35, 'name': 'Comedy'}]",tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...",...,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",16.326300,2015,3.448085,1,155
1,2,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]",...,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",18.370959,2004,9.960120,1,393
2,3,3300000,"[{'id': 18, 'name': 'Drama'}]",tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...",...,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",16.387512,2014,0.813570,1,130
8,9,0,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",tt0117110,en,Muppet Treasure Island,After telling the story of Flint's last journe...,6.902423,/5A8gKzOrF9Z7tSUX6xd5dEx4NXf.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...",...,Set sail for Muppet mayhem!,Muppet Treasure Island,"[{'id': 2041, 'name': 'island'}, {'id': 4418, ...","[{'cast_id': 1, 'character': 'Long John Silver...","[{'credit_id': '52fe43c89251416c7501deb3', 'de...",17.351454,1996,0.000000,1,397
9,10,6000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",tt0310281,en,A Mighty Wind,"In ""A Mighty Wind"", director Christopher Guest...",4.672036,/xwD4EsPOGLuKP50Q1gwqAsyhAFB.jpg,"[{'name': 'Castle Rock Entertainment', 'id': 97}]",...,"Back together for the first time, again.",A Mighty Wind,"[{'id': 11800, 'name': 'mockumentary'}, {'id':...","[{'cast_id': 24, 'character': 'Jonathan Steinb...","[{'credit_id': '52fe45609251416c750545b3', 'de...",16.746717,2003,1.495510,1,280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2994,2995,0,"[{'id': 18, 'name': 'Drama'}]",tt0105327,en,School Ties,When David Greene receives a football scholars...,7.438381,/poV3j71mcmQkmjezc2H35xJsAhD.jpg,"[{'name': 'Paramount Pictures', 'id': 4}]",...,Just because you’re accepted doesn’t mean you ...,School Ties,"[{'id': 6075, 'name': 'sport'}, {'id': 10144, ...","[{'cast_id': 2, 'character': 'David Greene', '...","[{'credit_id': '5637777ac3a3681b4d01f9f5', 'de...",16.504383,1992,0.000000,1,435
2995,2996,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",tt0109403,en,Chasers,Military men Rock Reilly and Eddie Devane are ...,9.853270,/j8Q7pQ27hvH54wpxJzIuQgQCdro.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",...,It was supposed to be a routine prisoner trans...,Chasers,"[{'id': 378, 'name': 'prison'}, {'id': 572, 'n...","[{'cast_id': 2, 'character': 'Rock Reilly', 'c...","[{'credit_id': '52fe4494c3a368484e02ac7d', 'de...",14.283442,1994,0.000000,1,261
2997,2998,65000000,"[{'id': 80, 'name': 'Crime'}, {'id': 28, 'name...",tt0116908,en,The Long Kiss Goodnight,"Samantha Caine, suburban homemaker, is the ide...",14.482345,/4MENR8x6mYqnZvp2hGjSaPJz64J.jpg,"[{'name': 'New Line Cinema', 'id': 12}, {'name...",...,What's forgotten is not always gone.,The Long Kiss Goodni

In [27]:
df['budget_runtime'] = (df['budget'] + 1)/(df['runtime']+1)
df['poularity_year'] = (df['popularity']+1)/(df['year']+1)
df['budget_popularity'] = (df['budget']+1)/(df['popularity']+1)
df['budget_year'] = (df['budget'] + 1)/(df['year']+1)
df['runtime_year'] = (df['runtime']+1)/(df['year']+1)

In [ ]:
# want to add 
# number of crew/cast
# Production companies 
# possibly add day of week, day of year, month, or quarter 
# number of keywords 
# has homepage 
# original language? 
# 

In [28]:
X = df[["popularity", "runtime", "budget", "budget_year_ratio", "year",'budget_runtime', 'poularity_year', 'budget_popularity','budget_year','runtime_year', 'has_overview', 'overview_len' ]]

In [29]:
X.head()

,popularity,runtime,budget,budget_year_ratio,year,budget_runtime,poularity_year,budget_popularity,budget_year,runtime_year,has_overview,overview_len
0,6.575393,93.0,14000000,3.448085,2015,148936.180851,0.003758,1.848089e+06,6944.444940,0.046627,1,155
1,8.248895,113.0,40000000,9.960120,2004,350877.201754,0.004613,4.324841e+06,19950.125187,0.056858,1,393
2,64.299990,105.0,3300000,0.813570,2014,31132.084906,0.032407,5.053601e+04,1637.717618,0.052605,1,130
8,6.902423,100.0,0,0.000000,1996,0.009901,0.003957,1.265435e-01,0.000501,0.050576,1,397
9,4.672036,91.0,6000000,1.495510,2003,65217.402174,0.002830,1.057821e+06,2994.012475,0.045908,1,280


In [30]:
#df['revenue'] = np.log1p(df['revenue'])
y = df[["revenue"]]

In [68]:
members = df["cast"]

num_cast = []

for i in members:
    if isinstance(i, str):
        number = sum(1 for c in i if c == "{")
        num_cast.append(number)
    else:
        num_cast.append(0)

df["num_cast"] = num_cast

In [74]:
genres = df["genres"]

first = []
second = []
third = []
fourth = []
fifth = []
sixth = []
seventh = []

for string in genres:
    if isinstance(string, str):
        splits = string.split("'")
        genre1 = splits[5]
        first.append(genre1)
        multiples1 = splits[7:]
        if not multiples1:
            second.append('NA')
        else:
            genre2 = multiples1[4]
            second.append(genre2)
        multiples2 = multiples1[7:]
        if not multiples2:
            third.append('NA')
        else:
            genre3 = multiples2[3]
            third.append(genre3)
        multiples3 = multiples2[6:]
        if not multiples3:
            fourth.append('NA')
        else:
            genre4 = multiples3[3]
            fourth.append(genre4)
        multiples4 = multiples3[5:]
        if not multiples4:
            fifth.append('NA')
        else:
            genre5 = multiples4[4]
            fifth.append(genre5)
        multiples5 = multiples4[6:]
        if not multiples5:
            sixth.append('NA')
        else:
            genre6 = multiples5[4]
            sixth.append(genre6)
        multiples6 = multiples5[6:]
        if not multiples6:
            seventh.append('NA')
        else:
            genre7 = multiples6[4]
            seventh.append(genre7)
    else:
        first.append('NA')
        second.append('NA')
        third.append('NA')
        fourth.append('NA')
        fifth.append('NA')
        sixth.append('NA')
        seventh.append('NA')

genres_list = list(zip(first, second, third, fourth, fifth, sixth, seventh))
df["genres_list"] = genres_list

In [75]:

genresInMovie = df["genres_list"]

action = []
adventure = []
animation = []
comedy = []
crime = []
documentary = []
drama = []
family = []
fantasy = []
foreign = []
horror = []
music = []
mystery = []
romance = []
scifi = []
thriller = []

for movie in genresInMovie:
    movie = list(movie)
    
    isAction = 0
    isAdventure = 0
    isAnimation = 0
    isComedy = 0
    isCrime = 0
    isDocumentary = 0
    isDrama = 0
    isFamily = 0
    isFantasy = 0
    isForeign = 0
    isHorror = 0
    isMusic = 0
    isMystery = 0
    isRomance = 0
    isSciFi = 0
    isThriller = 0
    
    for genre in movie:
        if genre == "Action":
            isAction = 1
        if genre == "Adventure":
            isAdventure = 1
        if genre == "Animation":
            isAnimation = 1
        if genre == "Comedy":
            isComedy = 1
        if genre == "Crime":
            isCrime = 1
        if genre == "Documentary":
            isDocumentary = 1
        if genre == "Drama":
            isDrama = 1
        if genre == "Family":
            isFamily = 1
        if genre == "Fantasy":
            isFantasy = 1
        if genre == "Foreign":
            isForeign = 1
        if genre == "Horror":
            isHorror = 1
        if genre == "Music":
            isMusic = 1
        if genre == "Mystery":
            isMystery = 1
        if genre == "Romance":
            isRomance = 1
        if genre == "Science Fiction":
            isSciFi = 1
        if genre == "Thriller":
            isThriller = 1
            
    action.append(isAction)
    adventure.append(isAdventure)
    animation.append(isAnimation)
    comedy.append(isComedy)
    crime.append(isCrime)
    documentary.append(isDocumentary)
    drama.append(isDrama)
    family.append(isFamily)
    fantasy.append(isFantasy)
    foreign.append(isForeign)
    horror.append(isHorror)
    music.append(isMusic)
    mystery.append(isMystery)
    romance.append(isRomance)
    scifi.append(isSciFi)
    thriller.append(isThriller)
    
df["isAction"] = action
df["isAdventure"] = adventure
df["isAnimation"] = animation
df["isComedy"] = comedy
df["isCrime"] = crime
df["isDocumentary"] = documentary
df["isDrama"] = drama
df["isFamily"] = family
df["isFantasy"] = fantasy
df["isForeign"] = foreign
df["isHorror"] = horror
df["isMusic"] = music
df["isMystery"] = mystery
df["isRomance"] = romance
df["isSciFi"] = scifi
df["isThriller"] = thriller

In [77]:
X = df[["popularity", "runtime", "budget", "budget_year_ratio", "year",'budget_runtime', 'poularity_year', 'budget_popularity','budget_year','runtime_year', 'has_overview', 'overview_len', "num_cast", "isAction", "isAdventure",
       "isAnimation","isComedy", "isCrime", "isDocumentary", "isDrama","isFamily", "isFantasy", "isForeign", 
       "isHorror", "isMusic", "isMystery","isRomance", "isSciFi", "isThriller"]]

In [76]:
df.head()

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,...,isDrama,isFamily,isFantasy,isForeign,isHorror,isMusic,isMystery,isRomance,isSciFi,isThriller
0,1,14000000,"[{'id': 35, 'name': 'Comedy'}]",tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...",...,0,0,0,0,0,0,0,0,0,0
1,2,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]",...,1,1,0,0,0,0,0,1,0,0
2,3,3300000,"[{'id': 18, 'name': 'Drama'}]",tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...",...,1,0,0,0,0,0,0,0,0,0
8,9,0,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",tt0117110,en,Muppet Treasure Island,After telling the story of Flint's last journe...,6.902423,/5A8gKzOrF9Z7tSUX6xd5dEx4NXf.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...",...,0,1,0,0,0,1,0,0,0,0
9,10,6000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",tt0310281,en,A Mighty Wind,"In ""A Mighty Wind"", director Christopher Guest...",4.672036,/xwD4EsPOGLuKP50Q1gwqAsyhAFB.jpg,"[{'name': 'Castle Rock Entertainment', 'id': 97}]",...,0,0,0,0,0,1,0,0,0,0


In [31]:
#X, y, nas = proc_df(df, "revenue")
#df_trn, y_trn, nas = proc_df(df_raw, 'revenue')

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [79]:
X_train.shape, X_test.shape, y_test.shape, y_train.shape

((1783, 29), (446, 29), (446, 1), (1783, 1))

In [34]:
'''
xgbr = xgb.XGBRegressor(verbosity=0) 
print(xgbr)
'''

'\nxgbr = xgb.XGBRegressor(verbosity=0) \nprint(xgbr)\n'

In [35]:
#xgbr.fit(X_train, y_train)

In [36]:
#ypred = xgbr.predict(X_test)
#mse = mean_squared_error(y_test, ypred)
#print("MSE: %.2f" % mse)
#MSE: 3.35
#print("RMSE: %.2f" % (mse**(1/2.0)))
#RMSE: 1.83 

In [82]:
m = RandomForestRegressor(n_estimators=40, random_state = 42, min_samples_leaf = 10,
                          n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)

C:\Users\Ben\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor(min_samples_leaf=10, n_estimators=40, n_jobs=-1,
                      oob_score=True, random_state=42)

In [59]:
def rmse(y_pred, y_true):
    return np.sqrt(mean_squared_error(y_pred, y_true))

def print_rf_score(model):
    print(f'Train R2:   {model.score(X_train, y_train)}')
    print(f'Test R2:   {model.score(X_test, y_test)}')
    print(f'Train RMSE: {rmse(model.predict(X_train), y_train)}')
    print(f'Test RMSE: {rmse(model.predict(X_test), y_test)}')

In [83]:
print_rf_score(m)

Train R2:   0.6479861470459249
Test R2:   0.5004332654910643
Train RMSE: 1.5421792518551893
Test RMSE: 1.8636598216244469


In [84]:
xgb = XGBRegressor(verbosity=2) 
xgb.fit(X_train, y_train)

[20:10:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:10:24] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[20:10:24] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[20:10:24] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[20:10:24] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[20:10:24] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[20:10:24] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[20:10:24] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[20:10:24] INFO: src/tree/updater_prune.cc:74: tree prunin

XGBRegressor(verbosity=2)

In [85]:
print_rf_score(xgb)

Train R2:   0.6790377882164096
Test R2:   0.49049230228327145
Train RMSE: 1.4725902835499465
Test RMSE: 1.8821111238733874
